# Map of italian fascism and right-wing extremism on Facebook pages

a collection of likes on facebook pages (not personal accounts!) of italian fascists and right-wing groups has been made and analyzed on this research by [ANPI](https://en.wikipedia.org/wiki/ANPI) (National Association of Italian Partisans):

http://patriaindipendente.it/progetto-facebook/  
http://patriaindipendente.it/progetto-facebook/doc.html

here i try to import this data into neo4j graph database to allow a more detailed discovery.


## import of data into neo4j

Requirements: neo4j, csvkit, jq

macos:

    brew install neo4j jq coreutils
    pip3 install csvkit


# download the raw data graph

the json is formatted to help reading

In [2]:
wget -q http://patriaindipendente.it/progetto-facebook/data.json.zip
unzip -q data.json.zip
jq . < data.json > graph.json
rm data.json*

the dataset is licensed under CC BY-NC-ND license [https://creativecommons.org/licenses/by-nc-nd/4.0/]

In [1]:
jq .license graph.json

"This dataset is available under CC BY-NC-ND license [https://creativecommons.org/licenses/by-nc-nd/4.0/], when using it you need to give proper credit to Giovanni Baldini <giovanni.baldini@anpi.it>. If interested in using it under a different license please contact the author."


# extract nodes

extract nodes from json and convert to csv.  
fix the header field.

In [4]:
jq .nodes graph.json | in2csv -f json > csv/nodes.csv
gsed -i '1 s/id/Node:ID/' csv/nodes.csv

`csv/nodes.csv` fields:

In [12]:
csvcut -n csv/nodes.csv

  1: load
  2: color
  3: pic
  4: tac
  5: in_deg
  6: Node:ID
  7: triangles
  8: clustering
  9: group
 10: fans
 11: label
 12: betweenness
 13: out_deg
 14: pagerank
 15: hits-hub
 16: x
 17: hits-authority
 18: u_clustering
 19: url
 20: p
 21: y
 22: eigenvector


In [11]:
head -n 3 csv/nodes.csv

load,color,pic,tac,in_deg,Node:ID,triangles,clustering,group,fans,label,betweenness,out_deg,pagerank,hits-hub,x,hits-authority,u_clustering,url,p,y,eigenvector
0,497173,https://scontent.xx.fbcdn.net/v/t1.0-1/c0.0.50.50/p50x50/10644938_650502445067413_6210163405274532119_n.jpg?oh=ced733b11686df3c242c17e42fa44011&oe=595CDCBF,0,1,650501465067511,0,0,-,0,Corrado Malanga in Suomen Kieli,0,0,0.00004631249396872088,0.00007250182370350222,1259.8876953125,0,0,/CorradoMalangaSuomi/,,1170.2913818359375,1.2093678122051513E-10
0,497173,https://scontent.xx.fbcdn.net/v/t1.0-1/p50x50/13516539_534569006746956_216770361829652764_n.jpg?oh=af759ad6c32a4d5699386c00f31a5611&oe=599ABA0A,0,3,164325450437982,3,1,-,0,Bashar Alassad,0,0,0.00004640058845995182,0.00025270053370682793,885.2281494140625,0,1,/DRbasharalassad/,,958.635009765625,8.539171155552731E-10


# extract edges

extract edges (relationships between nodes).  
fix the header field.

In [7]:
jq .edges graph.json | in2csv -f json | csvcut -c 1,3 | awk '{print $1",LIKE"}' > csv/edges.csv
gsed -i 1d csv/edges.csv
gsed -i '1i:END_ID,:START_ID,:TYPE' csv/edges.csv

`csv/edges.csv` fields:

In [13]:
csvcut -n csv/edges.csv

  1: :END_ID
  2: :START_ID
  3: :TYPE


In [14]:
head -n 5 csv/edges.csv

:END_ID,:START_ID,:TYPE
370198643136742,535069289865058,LIKE
494189513998338,393977967466157,LIKE
201275493224544,833352813372167,LIKE
599831756702393,636934606389960,LIKE


# import into neo4j

import the csv into neo4j

In [9]:
neo4j-import --into graph.db --nodes:Node csv/nodes.csv  --relationships:LIKE csv/edges.csv

version of Neo4j; please use neo4j-admin import instead.

Neo4j version: 3.3.0
Importing the contents of these files into graph.db:
Nodes:
  :Node
  /Users/raffaele/fascists-facebook-graph/csv/nodes.csv
Relationships:
  :LIKE
  /Users/raffaele/fascists-facebook-graph/csv/edges.csv

Available resources:
  Total machine memory: 16.00 GB
  Free machine memory: 4.89 GB
  Max heap memory : 3.56 GB
  Processors: 4
  Configured max memory: 1.20 GB

Nodes, started 2017-11-17 09:19:33.434+0000
[*>:??--------------------------------|NODE:7.|PROPERTIES(2)===============||v:??(2)==========]16.3K ∆16.3K
Done in 934ms
Prepare node index, started 2017-11-17 09:19:34.462+0000
[*DETECT:7.69 MB------------------------------------------------------------------------------]10.0K ∆10.0K
Done in 97ms
Relationships, started 2017-11-17 09:19:34.585+0000
[*>:??------------------------------------|TY|PREPARE(2)========================|RECOR|P|v:??-]54.6K ∆54.6K
Done in 454ms
Node Degrees, started 2017-11-17 09:

## replace neo4j default database

In [15]:
rm -fr /usr/local/Cellar/neo4j/3.3.0/libexec/data/databases/graph.db

In [16]:
mv graph.db /usr/local/Cellar/neo4j/3.3.0/libexec/data/databases/

# start neo4j

    $ neo4j start

# simple queries in cypher

1) search nodes label containing a string:

In [6]:
cypher-shell --format plain 'MATCH (n) WHERE n.label CONTAINS "Forza Nuova" RETURN n.label, n.url LIMIT 20'

n.label, n.url
"Forza Nuova Sardegna", "/forza.nuova.sardegna/"
"Forza Nuova Disabilità", "/Forza-Nuova-Disabilità-1563170127292287/"
"Forza Nuova Borgomanero", "/Forza-Nuova-Borgomanero-174543319382224/"
"Forza Nuova Paternò - CT", "/ForzaNuovaPaterno/"
"Forza Nuova Biella", "/Forza-Nuova-Biella-732281886816134/"
"Forza Nuova Livorno", "/ForzaNuovaProvinciaLivorno/"
"Forza Nuova Merano", "/forzanuova.info/"
"Forza Nuova Nova Milanese", "/forzanuova.novamilanese/"
"Forza Nuova Eboli - Campagna", "/ForzaNuovaEboli/"
"Forza Nuova Agropoli", "/ForzaNuovaAgropoli/"
"Forza Nuova Roseto", "/Forza-Nuova-Roseto-435464570179127/"
"Forza Nuova Manerbio", "/forzanuova.Manerbio/"
"Forza Nuova Bressanone - Brixen", "/ForzaNuovaBressanone/"
"Forza Nuova Ciriè", "/GabriFn.Nasi/"
"Forza Nuova Taranto", "/ForzaNuovaTaranto/"
"Forza Nuova Padova", "/Forza-Nuova-Padova-225234430878587/"
"Forza Nuova Alba", "/Forza-Nuova-Alba-496842513729908/"
"Forza Nuova Orta Nova", "/ForzaNuovaOrtanovaStornaraStornarel

2) top 10 likers:

In [3]:
cypher-shell \
"MATCH (node)-[like]->()
RETURN node.label, COUNT(like)
ORDER BY COUNT(like) DESC
LIMIT 10"

+-------------------------------------------------------------+
| node.label                                    | COUNT(like) |
+-------------------------------------------------------------+
| "Interforze Cosmo1"                           | 4207        |
| "NuovaForza Monteclarense"                    | 2290        |
| "Destra Nazionalista"                         | 2250        |
| "Opposizione Popolare"                        | 965         |
| "Pensiero indipendente"                       | 633         |
| "Nessuna Resa"                                | 561         |
| "Gioventù Maceratese"                         | 452         |
| "Rock Nazionalista"                           | 419         |
| "Faro Francesco Crispi 1924 - Capo Guardafui" | 320         |
| "Voice Of A Generation - Skinzine"            | 306         |
+-------------------------------------------------------------+

10 rows available after 122 ms, consumed after another 0 ms


3) top 10 liked:

In [4]:
cypher-shell \
"MATCH ()-[liked]->(node)
RETURN node.label, COUNT(liked)
ORDER BY COUNT(liked) DESC
LIMIT 10"

+---------------------------------------+
| node.label             | COUNT(liked) |
+---------------------------------------+
| "Forza Nuova"          | 421          |
| "CasaPound Italia"     | 420          |
| "Roberto Fiore"        | 197          |
| "Blocco Studentesco"   | 192          |
| "Lotta Studentesca"    | 189          |
| "Simone Di Stefano"    | 173          |
| "Gianluca Iannone"     | 167          |
| "Il Primato Nazionale" | 167          |
| "Radio Bandiera Nera"  | 163          |
| "Giorgio Almirante"    | 154          |
+---------------------------------------+

10 rows available after 133 ms, consumed after another 0 ms
